# Week 14 — Data Analytics & SQL Evaluation
### BenchRight LLM Evaluation Master Program (18 Weeks)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. Understand how to evaluate LLM SQL generation capabilities
2. Create an in-memory SQLite database for testing
3. Implement a two-metric evaluation: execution success and result correctness
4. Analyze which types of queries are hardest for models to generate
5. Build a complete text-to-SQL evaluation pipeline

---

## 🧠 Why SQL Evaluation is Different

### The Challenge

Unlike natural language tasks, SQL has **objective correctness criteria**:

| Aspect | Natural Language | SQL |
|--------|------------------|-----|
| Correctness | Multiple phrasings OK | Must execute AND return correct results |
| Evaluation | Human judgment needed | Automated testing possible |
| Errors | Graceful degradation | Syntax error = complete failure |

### What We Evaluate

1. **Execution Success:** Does the query run without errors?
2. **Result Correctness:** Does the query return the expected answer?
3. **Schema Understanding:** Does the model correctly reference tables and columns?

---

## 🛠️ Step 1: Setup & Dependencies

In [ ]:
# Standard library imports
import sqlite3
import sys
import json
from typing import Dict, List, Any, Tuple, Optional, Callable

# Add src to path if running in Colab
sys.path.insert(0, '.')

# For data display
try:
    from IPython.display import display, HTML
except ImportError:
    display = print

print("✅ Setup complete!")
print(f"   SQLite version: {sqlite3.sqlite_version}")

---

## 🗄️ Step 2: Create the In-Memory SQLite Database

In [ ]:
# Sales analytics database schema
SCHEMA_SQL = """
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT,
    city TEXT,
    signup_date DATE
);

CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    category TEXT,
    price REAL
);

CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    total_amount REAL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

CREATE TABLE order_items (
    item_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    unit_price REAL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
"""

# Sample data for the database
DATA_SQL = """
INSERT INTO customers VALUES (1, 'Alice Johnson', 'alice@example.com', 'New York', '2023-01-15');
INSERT INTO customers VALUES (2, 'Bob Smith', 'bob@example.com', 'Los Angeles', '2023-02-20');
INSERT INTO customers VALUES (3, 'Charlie Brown', 'charlie@example.com', 'New York', '2023-03-10');
INSERT INTO customers VALUES (4, 'Diana Lee', 'diana@example.com', 'Chicago', '2024-01-05');
INSERT INTO customers VALUES (5, 'Eve Wilson', 'eve@example.com', 'Los Angeles', '2024-02-15');

INSERT INTO products VALUES (1, 'Laptop', 'Electronics', 999.99);
INSERT INTO products VALUES (2, 'Headphones', 'Electronics', 149.99);
INSERT INTO products VALUES (3, 'Coffee Maker', 'Appliances', 79.99);
INSERT INTO products VALUES (4, 'Desk Chair', 'Furniture', 249.99);
INSERT INTO products VALUES (5, 'Monitor', 'Electronics', 399.99);

INSERT INTO orders VALUES (1, 1, '2024-01-10', 1149.98);
INSERT INTO orders VALUES (2, 1, '2024-02-15', 79.99);
INSERT INTO orders VALUES (3, 2, '2024-01-20', 399.99);
INSERT INTO orders VALUES (4, 3, '2024-02-01', 249.99);
INSERT INTO orders VALUES (5, 4, '2024-03-01', 549.98);
INSERT INTO orders VALUES (6, 2, '2024-03-15', 149.99);

INSERT INTO order_items VALUES (1, 1, 1, 1, 999.99);
INSERT INTO order_items VALUES (2, 1, 2, 1, 149.99);
INSERT INTO order_items VALUES (3, 2, 3, 1, 79.99);
INSERT INTO order_items VALUES (4, 3, 5, 1, 399.99);
INSERT INTO order_items VALUES (5, 4, 4, 1, 249.99);
INSERT INTO order_items VALUES (6, 5, 2, 2, 149.99);
INSERT INTO order_items VALUES (7, 5, 4, 1, 249.99);
INSERT INTO order_items VALUES (8, 6, 2, 1, 149.99);
"""

print("📊 Database schema and sample data defined!")
print("")
print("Tables:")
print("  • customers (5 rows)")
print("  • products (5 rows)")
print("  • orders (6 rows)")
print("  • order_items (8 rows)")

---

## 🧪 Step 3: Implement the SQLEvaluator Class

In [ ]:
class SQLEvaluator:
    """
    Evaluator for text-to-SQL tasks.
    
    Uses an in-memory SQLite database to verify:
    1. Query execution success
    2. Result correctness (compared to reference answer)
    """
    
    def __init__(self, schema_sql: str, data_sql: str):
        """
        Initialize the SQLEvaluator with a database schema and data.
        
        Args:
            schema_sql: SQL statements to create tables
            data_sql: SQL statements to insert sample data
        """
        self.schema_sql = schema_sql
        self.data_sql = data_sql
        self._init_database()
    
    def _init_database(self) -> None:
        """Initialize the in-memory SQLite database."""
        self.conn = sqlite3.connect(':memory:')
        self.cursor = self.conn.cursor()
        
        # Create schema
        for statement in self.schema_sql.split(';'):
            statement = statement.strip()
            if statement:
                self.cursor.execute(statement)
        
        # Insert data
        for statement in self.data_sql.split(';'):
            statement = statement.strip()
            if statement:
                self.cursor.execute(statement)
        
        self.conn.commit()
    
    def execute_query(self, sql: str) -> Dict[str, Any]:
        """
        Execute a SQL query and return results.
        
        Args:
            sql: SQL query to execute
            
        Returns:
            Dictionary with:
            - success: bool indicating if query executed
            - result: query results if successful
            - columns: column names
            - error: error message if failed
        """
        try:
            self.cursor.execute(sql)
            results = self.cursor.fetchall()
            columns = [desc[0] for desc in self.cursor.description] if self.cursor.description else []
            
            return {
                "success": True,
                "result": results,
                "columns": columns,
                "error": None,
            }
        except Exception as e:
            return {
                "success": False,
                "result": None,
                "columns": [],
                "error": str(e),
            }
    
    def compare_results(
        self,
        generated_result: List[Tuple],
        reference_result: List[Tuple],
        order_matters: bool = False,
    ) -> bool:
        """
        Compare generated query results with reference results.
        
        Args:
            generated_result: Results from generated query
            reference_result: Expected reference results
            order_matters: Whether row order should match
            
        Returns:
            True if results match, False otherwise
        """
        if generated_result is None or reference_result is None:
            return False
        
        if order_matters:
            return generated_result == reference_result
        else:
            # Compare as sets of tuples
            return set(generated_result) == set(reference_result)
    
    def evaluate_query(
        self,
        generated_sql: str,
        reference_sql: str,
        order_matters: bool = False,
    ) -> Dict[str, Any]:
        """
        Evaluate a generated SQL query.
        
        Args:
            generated_sql: The SQL query to evaluate
            reference_sql: Reference SQL with correct answer
            order_matters: Whether result order should match
            
        Returns:
            Dictionary with:
            - execution_success: bool
            - result_match: bool
            - generated_result: results from generated query
            - reference_result: results from reference query
            - error: error message if any
        """
        # Execute reference query to get expected result
        ref_execution = self.execute_query(reference_sql)
        if not ref_execution["success"]:
            return {
                "execution_success": False,
                "result_match": False,
                "generated_result": None,
                "reference_result": None,
                "error": f"Reference query failed: {ref_execution['error']}",
            }
        
        # Execute generated query
        gen_execution = self.execute_query(generated_sql)
        
        if not gen_execution["success"]:
            return {
                "execution_success": False,
                "result_match": False,
                "generated_result": None,
                "reference_result": ref_execution["result"],
                "error": gen_execution["error"],
            }
        
        # Compare results
        results_match = self.compare_results(
            gen_execution["result"],
            ref_execution["result"],
            order_matters=order_matters,
        )
        
        return {
            "execution_success": True,
            "result_match": results_match,
            "generated_result": gen_execution["result"],
            "reference_result": ref_execution["result"],
            "error": None,
        }
    
    def compute_metrics(
        self, 
        results: List[Dict[str, Any]],
    ) -> Dict[str, float]:
        """
        Compute aggregate metrics across multiple evaluations.
        
        Args:
            results: List of evaluation results
            
        Returns:
            Dictionary with:
            - execution_rate: proportion of queries that executed
            - accuracy: proportion of queries with correct results
        """
        if not results:
            return {"execution_rate": 0.0, "accuracy": 0.0}
        
        executed = sum(1 for r in results if r["execution_success"])
        correct = sum(1 for r in results if r["result_match"])
        
        return {
            "execution_rate": executed / len(results),
            "accuracy": correct / len(results),
        }
    
    def close(self) -> None:
        """Close the database connection."""
        self.conn.close()


print("✅ SQLEvaluator class defined!")

---

## 🏃 Step 4: Initialize the Evaluator

In [ ]:
# Create the SQLEvaluator
evaluator = SQLEvaluator(SCHEMA_SQL, DATA_SQL)

print("✅ SQLEvaluator initialized with in-memory database!")
print("")

# Verify database is working
test_result = evaluator.execute_query("SELECT COUNT(*) FROM customers")
print(f"Test query result: {test_result['result'][0][0]} customers in database")

---

## 📋 Step 5: Define Text-to-SQL Test Cases

In [ ]:
# Comprehensive test cases with varying difficulty
TEST_CASES = [
    # Easy: Simple queries
    {
        "name": "Simple Count",
        "question": "How many customers are there in total?",
        "reference_sql": "SELECT COUNT(*) FROM customers",
        "difficulty": "easy",
    },
    {
        "name": "Filter by City",
        "question": "What are the names of customers from New York?",
        "reference_sql": "SELECT name FROM customers WHERE city = 'New York'",
        "difficulty": "easy",
    },
    {
        "name": "Sum Aggregation",
        "question": "What is the total revenue from all orders?",
        "reference_sql": "SELECT SUM(total_amount) FROM orders",
        "difficulty": "easy",
    },
    {
        "name": "Average Calculation",
        "question": "What is the average product price?",
        "reference_sql": "SELECT AVG(price) FROM products",
        "difficulty": "easy",
    },
    # Medium: Queries with joins or grouping
    {
        "name": "Group By with Count",
        "question": "How many customers are there in each city?",
        "reference_sql": "SELECT city, COUNT(*) FROM customers GROUP BY city",
        "difficulty": "medium",
    },
    {
        "name": "Simple Join",
        "question": "List the names of customers who have placed orders.",
        "reference_sql": "SELECT DISTINCT c.name FROM customers c JOIN orders o ON c.customer_id = o.customer_id",
        "difficulty": "medium",
    },
    {
        "name": "Date Filtering",
        "question": "How many orders were placed in 2024?",
        "reference_sql": "SELECT COUNT(*) FROM orders WHERE order_date >= '2024-01-01'",
        "difficulty": "medium",
    },
    # Hard: Complex multi-table queries
    {
        "name": "Group By with Sum and Join",
        "question": "What is the total sales per product category?",
        "reference_sql": "SELECT p.category, SUM(oi.quantity * oi.unit_price) FROM order_items oi JOIN products p ON oi.product_id = p.product_id GROUP BY p.category",
        "difficulty": "hard",
    },
    {
        "name": "Subquery",
        "question": "Which customers have placed more than one order?",
        "reference_sql": "SELECT name FROM customers WHERE customer_id IN (SELECT customer_id FROM orders GROUP BY customer_id HAVING COUNT(*) > 1)",
        "difficulty": "hard",
    },
    {
        "name": "Complex Aggregation with Order",
        "question": "What is the most expensive product in the Electronics category?",
        "reference_sql": "SELECT name, price FROM products WHERE category = 'Electronics' ORDER BY price DESC LIMIT 1",
        "difficulty": "hard",
    },
]

print(f"📋 Defined {len(TEST_CASES)} test cases:")
print("")
for difficulty in ["easy", "medium", "hard"]:
    cases = [tc for tc in TEST_CASES if tc["difficulty"] == difficulty]
    print(f"  {difficulty.upper()}: {len(cases)} cases")
    for tc in cases:
        print(f"    • {tc['name']}")

---

## ✓ Step 6: Verify Reference Queries Execute Correctly

First, let's verify that all our reference SQL queries work as expected.

In [ ]:
print("🔍 Verifying Reference Queries...")
print("=" * 70)

verification_results = []
for tc in TEST_CASES:
    result = evaluator.execute_query(tc["reference_sql"])
    
    verification_results.append({
        "name": tc["name"],
        "difficulty": tc["difficulty"],
        "success": result["success"],
        "result": result["result"],
        "error": result["error"],
    })
    
    status = "✅ Success" if result["success"] else "❌ Failed"
    print(f"\n{status} [{tc['difficulty']}] {tc['name']}")
    print(f"   Question: {tc['question']}")
    print(f"   SQL: {tc['reference_sql']}")
    if result["success"]:
        print(f"   Result: {result['result']}")
    else:
        print(f"   Error: {result['error']}")

# Summary
success_count = sum(1 for r in verification_results if r["success"])
print("\n" + "=" * 70)
print(f"📊 Reference Query Verification: {success_count}/{len(TEST_CASES)} successful")
if success_count == len(TEST_CASES):
    print("✅ All reference queries execute successfully!")

---

## 🤖 Step 7: Define Mock Model for Demonstration

In [ ]:
class MockSQLModel:
    """
    Mock model that simulates LLM SQL generation responses.
    
    For demonstration, it returns correct SQL for some queries
    and intentionally incorrect SQL for others to show evaluation.
    """
    
    def __init__(self):
        """
        Initialize the mock model.
        
        Predefine responses for each test case.
        Some are correct, some have intentional errors.
        """
        # Map question patterns to SQL responses
        # Some correct, some with intentional errors
        self.responses = {
            # Correct responses
            "How many customers are there in total?": 
                "SELECT COUNT(*) FROM customers",
            "What are the names of customers from New York?": 
                "SELECT name FROM customers WHERE city = 'New York'",
            "What is the total revenue from all orders?": 
                "SELECT SUM(total_amount) FROM orders",
            "What is the average product price?": 
                "SELECT AVG(price) FROM products",
            "How many customers are there in each city?": 
                "SELECT city, COUNT(*) FROM customers GROUP BY city",
            "How many orders were placed in 2024?": 
                "SELECT COUNT(*) FROM orders WHERE order_date >= '2024-01-01'",
            
            # Intentionally incorrect responses for demonstration
            "List the names of customers who have placed orders.": 
                "SELECT c.name FROM customers c, orders o",  # Missing join condition - Cartesian product
            "What is the total sales per product category?": 
                "SELECT category, SUM(price) FROM products GROUP BY category",  # Wrong table/calculation
            "Which customers have placed more than one order?": 
                "SELECT name FROM customers WHERE customer_id IN (SELECT customer_id FROM orders)",  # Missing HAVING
            "What is the most expensive product in the Electronics category?": 
                "SELECT name FROM products WHERE category = 'Electronics' ORDER BY price",  # Missing DESC and LIMIT
        }
    
    def generate_sql(self, question: str) -> str:
        """
        Generate SQL for a given question.
        
        Args:
            question: Natural language question
            
        Returns:
            Generated SQL query
        """
        # Look for matching question
        for q, sql in self.responses.items():
            if question.strip() == q:
                return sql
        
        # Default: return a syntax error query
        return "SELECT * FORM broken_table"  # Intentional typo


# Create mock model
mock_model = MockSQLModel()
print("✅ Mock SQL model created!")
print("   (Simulates varying model performance for demonstration)")
print("")
print("   Correct responses: 6 queries")
print("   Incorrect responses: 4 queries")

---

## 📝 Step 8: Define Text-to-SQL Prompt Template

In [ ]:
# Schema description for prompts
SCHEMA_DESCRIPTION = """
Tables:
- customers (customer_id, name, email, city, signup_date)
- products (product_id, name, category, price)
- orders (order_id, customer_id, order_date, total_amount)
- order_items (item_id, order_id, product_id, quantity, unit_price)

Relationships:
- orders.customer_id references customers.customer_id
- order_items.order_id references orders.order_id
- order_items.product_id references products.product_id
"""

TEXT_TO_SQL_PROMPT = """You are a SQL expert. Given the following database schema and question, write a SQL query that answers the question.

## Database Schema
{schema}

## Question
{question}

## Instructions
1. Write a valid SQLite query
2. Return ONLY the SQL query, no explanations
3. Do not include markdown code blocks

## SQL Query:
"""


def create_text_to_sql_prompt(question: str) -> str:
    """Create a prompt for text-to-SQL tasks."""
    return TEXT_TO_SQL_PROMPT.format(
        schema=SCHEMA_DESCRIPTION,
        question=question,
    )


print("✅ Prompt template defined!")
print("")
print("Example prompt:")
print("-" * 40)
print(create_text_to_sql_prompt("How many customers are there?")[:300] + "...")

---

## 🏃 Step 9: Run the Evaluation Pipeline

In [ ]:
print("🔄 Running Text-to-SQL Evaluation...")
print("=" * 70)

evaluation_results = []

for tc in TEST_CASES:
    # Generate SQL using mock model
    generated_sql = mock_model.generate_sql(tc["question"])
    
    # Evaluate the generated SQL
    result = evaluator.evaluate_query(
        generated_sql=generated_sql,
        reference_sql=tc["reference_sql"],
        order_matters=False,
    )
    
    evaluation_results.append({
        "name": tc["name"],
        "question": tc["question"],
        "difficulty": tc["difficulty"],
        "generated_sql": generated_sql,
        "reference_sql": tc["reference_sql"],
        **result,
    })
    
    # Display results
    exec_status = "✅" if result["execution_success"] else "❌"
    match_status = "✅" if result["result_match"] else "❌"
    
    print(f"\n[{tc['difficulty'].upper()}] {tc['name']}")
    print(f"   Question: {tc['question']}")
    print(f"   Generated: {generated_sql}")
    print(f"   Reference: {tc['reference_sql']}")
    print(f"   Execution: {exec_status} | Result Match: {match_status}")
    if result["error"]:
        print(f"   Error: {result['error']}")
    if result["execution_success"] and not result["result_match"]:
        print(f"   Generated Result: {result['generated_result']}")
        print(f"   Expected Result: {result['reference_result']}")

print("\n" + "=" * 70)

---

## 📊 Step 10: Compute and Display Metrics

In [ ]:
# Compute overall metrics
metrics = evaluator.compute_metrics(evaluation_results)

print("📊 Overall Evaluation Metrics")
print("=" * 70)
print(f"")
print(f"Total Test Cases: {len(evaluation_results)}")
print(f"Execution Rate: {metrics['execution_rate']:.0%}")
print(f"Accuracy (Result Match): {metrics['accuracy']:.0%}")
print("")

# Compute metrics by difficulty
print("📈 Metrics by Difficulty")
print("-" * 40)

for difficulty in ["easy", "medium", "hard"]:
    difficulty_results = [r for r in evaluation_results if r["difficulty"] == difficulty]
    if difficulty_results:
        diff_metrics = evaluator.compute_metrics(difficulty_results)
        print(f"")
        print(f"{difficulty.upper()} ({len(difficulty_results)} queries):")
        print(f"   Execution Rate: {diff_metrics['execution_rate']:.0%}")
        print(f"   Accuracy: {diff_metrics['accuracy']:.0%}")

---

## 📋 Step 11: Generate Summary Table

In [ ]:
print("📋 Evaluation Summary Table")
print("=" * 90)
print(f"{'#':<3} {'Name':<30} {'Difficulty':<10} {'Execution':<12} {'Result Match':<12}")
print("-" * 90)

for i, r in enumerate(evaluation_results, 1):
    exec_status = "✅ Pass" if r["execution_success"] else "❌ Fail"
    match_status = "✅ Match" if r["result_match"] else "❌ Mismatch"
    
    print(f"{i:<3} {r['name']:<30} {r['difficulty']:<10} {exec_status:<12} {match_status:<12}")

print("-" * 90)
print(f"")
print(f"Summary: {metrics['accuracy']:.0%} accuracy across {len(evaluation_results)} test cases")

---

## 🔍 Step 12: Analyze Failure Patterns

In [ ]:
# Analyze failures
execution_failures = [r for r in evaluation_results if not r["execution_success"]]
result_mismatches = [r for r in evaluation_results if r["execution_success"] and not r["result_match"]]
successes = [r for r in evaluation_results if r["result_match"]]

print("🔍 Failure Analysis")
print("=" * 70)

print(f"")
print(f"✅ Successful: {len(successes)} queries")
print(f"❌ Execution Failures: {len(execution_failures)} queries")
print(f"⚠️ Result Mismatches: {len(result_mismatches)} queries")

if execution_failures:
    print("")
    print("❌ Execution Failures:")
    print("-" * 40)
    for r in execution_failures:
        print(f"   • {r['name']}: {r['error'][:50]}...")

if result_mismatches:
    print("")
    print("⚠️ Result Mismatches:")
    print("-" * 40)
    for r in result_mismatches:
        print(f"   • {r['name']}")
        print(f"     Generated: {r['generated_result']}")
        print(f"     Expected:  {r['reference_result']}")

---

## 🧪 Step 13: Test with Custom Queries

Try your own SQL queries to see the evaluation in action!

In [ ]:
# Test custom queries
print("🧪 Custom Query Testing")
print("=" * 70)

# Example: Test equivalent queries that return same results
custom_tests = [
    {
        "name": "Equivalent COUNT queries",
        "generated": "SELECT COUNT(customer_id) FROM customers",
        "reference": "SELECT COUNT(*) FROM customers",
    },
    {
        "name": "Different column alias",
        "generated": "SELECT name AS customer_name FROM customers WHERE city = 'New York'",
        "reference": "SELECT name FROM customers WHERE city = 'New York'",
    },
    {
        "name": "Wrong filter value",
        "generated": "SELECT name FROM customers WHERE city = 'Boston'",
        "reference": "SELECT name FROM customers WHERE city = 'New York'",
    },
]

for test in custom_tests:
    result = evaluator.evaluate_query(
        generated_sql=test["generated"],
        reference_sql=test["reference"],
    )
    
    exec_status = "✅" if result["execution_success"] else "❌"
    match_status = "✅" if result["result_match"] else "❌"
    
    print(f"\n{test['name']}")
    print(f"   Generated: {test['generated']}")
    print(f"   Reference: {test['reference']}")
    print(f"   Execution: {exec_status} | Result Match: {match_status}")
    if result["execution_success"]:
        print(f"   Results: {result['generated_result']} vs {result['reference_result']}")

---

## 🎯 Step 14: Query Complexity Scoring

In [ ]:
def score_query_complexity(sql: str) -> Dict[str, Any]:
    """
    Score the complexity of a SQL query.
    
    Returns:
        Dictionary with:
        - complexity_score: 1-10 scale
        - features: list of detected features
        - difficulty: "easy", "medium", "hard"
    """
    sql_upper = sql.upper()
    
    features = []
    score = 1
    
    # Check for various SQL features
    if "JOIN" in sql_upper:
        features.append("JOIN")
        score += 2
    
    if "GROUP BY" in sql_upper:
        features.append("GROUP BY")
        score += 2
    
    if "HAVING" in sql_upper:
        features.append("HAVING")
        score += 1
    
    if sql.upper().count("SELECT") > 1:
        features.append("SUBQUERY")
        score += 3
    
    if "UNION" in sql_upper:
        features.append("UNION")
        score += 2
    
    if "ORDER BY" in sql_upper:
        features.append("ORDER BY")
        score += 1
    
    if sql.upper().count("JOIN") > 1:
        features.append("MULTIPLE_JOINS")
        score += 2
    
    if "WHERE" in sql_upper:
        features.append("WHERE")
        score += 1
    
    # Determine difficulty
    if score <= 2:
        difficulty = "easy"
    elif score <= 5:
        difficulty = "medium"
    else:
        difficulty = "hard"
    
    return {
        "complexity_score": min(score, 10),
        "features": features,
        "difficulty": difficulty,
    }


print("🎯 Query Complexity Analysis")
print("=" * 70)

for tc in TEST_CASES:
    complexity = score_query_complexity(tc["reference_sql"])
    print(f"\n{tc['name']}")
    print(f"   SQL: {tc['reference_sql'][:50]}..." if len(tc['reference_sql']) > 50 else f"   SQL: {tc['reference_sql']}")
    print(f"   Score: {complexity['complexity_score']}/10 ({complexity['difficulty']})")
    print(f"   Features: {', '.join(complexity['features']) if complexity['features'] else 'Basic SELECT'}")

---

## 🧹 Step 15: Cleanup

In [ ]:
# Close the database connection
evaluator.close()

print("✅ Database connection closed.")
print("")
print("🎉 Week 14 Lab Complete!")
print("")
print("Key Takeaways:")
print("  1. SQL evaluation uses two metrics: execution success + result correctness")
print("  2. In-memory SQLite provides fast, isolated testing")
print("  3. Different query types have varying difficulty levels")
print("  4. Result comparison can be order-sensitive or order-insensitive")
print("  5. Query complexity scoring helps categorize test cases")

---

## 📚 Summary

In this notebook, you learned how to:

1. **Create an in-memory SQLite database** for testing SQL generation
2. **Implement the SQLEvaluator class** with execution and result comparison
3. **Define test cases** with varying difficulty levels
4. **Run evaluations** on generated SQL queries
5. **Compute metrics** including execution rate and accuracy
6. **Analyze failures** to understand model weaknesses
7. **Score query complexity** to categorize evaluation difficulty

### Next Steps

1. **Extend the database schema** with more complex relationships
2. **Add more test cases** covering edge cases and advanced SQL features
3. **Integrate with a real LLM** instead of the mock model
4. **Implement semantic equivalence** for more flexible result comparison
5. **Add query performance metrics** (execution time, resource usage)

---

## ✔ Knowledge Mastery Checklist

Before moving to Week 15, ensure you can check all boxes:

- [ ] I understand why SQL evaluation requires both execution testing and result comparison
- [ ] I can create an in-memory SQLite database for testing SQL generation
- [ ] I can use the SQLEvaluator to test generated queries
- [ ] I understand the two-metric approach: execution success and result correctness
- [ ] I can design test cases with varying difficulty levels
- [ ] I know how to handle result comparison (order-sensitive vs. order-insensitive)
- [ ] I can analyze failure patterns in SQL generation
- [ ] I understand query complexity scoring

---

**Week 14 Complete!**

*Next: Week 15 — RAG (Retrieval-Augmented Generation) Use Cases*